# Sequence Classification
 

### Imports

In [2]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
batch_size = 64
learning_rate = 0.001
training_epochs = 5

Mounted at /content/drive
cuda


### 1 Dataset



In [3]:
# IMDB Dataset

# data.Field 설명 #
# sequential인자 : TEXT는 Sequential 데이터라 True, Lable은 비Sequential이라 False로 설정
# batch_first : Batch를 우선시 하여, Tensor 크기를 (BATCH_SIZE, 문장의 최대 길이)로 설정
# lower : 소문자 전환 인자

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

# data.Field.build_vocab() 라이브러리
# 문장 내 단어와 Integer index 를 매칭시키는 단어장(vocab)을 생성 == 워드 임베딩을 위한 Vocab 생성
# <UNK> = 0, <PAD> = 1 토큰도 추가.
# min_freq : 최소 5번 이상 등장한 단어들만 사전에 담겠다는 것. 
# 5번 미만으로 등장하는 단어는 UNK라는 토큰으로 대체

TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
trainset, valset = trainset.split(split_ratio=0.8)
# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=batch_size,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 2 # Positive, Negative Class가 두 개

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:02<00:00, 41.4MB/s]


[TrainSet]: 20000 [ValSet]: 5000 [TestSet]: 25000 [Vocab]: 46159 [Classes] 2


### 2 RNN model
* Layer 설계
  + Layer 1
    - Embedding Layer
    - Input size = n_Vocabs = 46159
    - Output size = Embedding size
  + Layer 2
    - GRU Layer
    - Input size = Embedding size
    - Output size = Hidden size
    - Dropout = 0.2
  + Layer 3
    - Linear Layer
    - Input size = Hidden size
    - Output size = n_Classes = 2

In [5]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        self.n_layers = n_layers # 일반적으로는 2

        #n_vocab : Vocab 안에 있는 단어의 개수, embed_dim : 임베딩 된 단어 텐서가 갖는 차원 값(dimension)
        self.embed = nn.Embedding(n_vocab, embed_dim)

        # hidden vector의 dimension과 dropout 정의
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)

        #앞에서 정의한 하이퍼 파라미터를 넣어 GRU 정의
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        
        #Input: GRU의 hidden vector(context), Output : Class probability vector
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        # Input data: 한 batch 내 모든 영화 평가 데이터
        
        x = self.embed(x)# 영화 평 임베딩
        x, _ = self.gru(x)  # [i, b, h] 출력값 :  (batch_size, 입력 x의 길이, hidden_dim)

        # h_t : Batch 내 모든 sequential hidden state vector의 제일 마지막 토큰을 내포한 (batch_size, 1, hidden_dim)형태의 텐서 추출
        # 다른 의미로 영화 리뷰 배열들을 압축한 hidden state vector
        h_t = x[:,-1,:]

        self.dropout(h_t)# dropout 설정 후, 

        # linear layer의 입력으로 주고, 각 클래스 별 결과 logit을 생성.
        out = self.out(h_t)  # [b, h] -> [b, o]
        return out

### 3 학습하기

In [6]:
# contruct model
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
for epoch in range(training_epochs):
  avg_cost = 0
  for batch in train_iter:
    X, Y = batch.text.to(device), batch.label.to(device)
    Y.data.sub_(1)
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / batch_size
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

# model save
torch.save(model.state_dict(), '/content/drive/MyDrive/model_s1.pt')

[Epoch:    1] cost = 3.41189408
[Epoch:    2] cost = 3.39696717
[Epoch:    3] cost = 3.39243889
[Epoch:    4] cost = 3.38118362
[Epoch:    5] cost = 3.14284444
Learning Finished!


In [8]:
# model load
model_new = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
model_new.load_state_dict(torch.load('/content/drive/MyDrive/model_s1.pt'))

corrects = 0
for batch in val_iter:
  x,y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  hypothesis = model_new(x)
  corrects += (hypothesis.max(1)[1].view(y.size()).data == y.data).sum() 

print('accuracy = ', corrects/len(val_iter.dataset)*100.0)

accuracy =  tensor(77.8000, device='cuda:0')


### 4 Assignment
### a) 아래 예제 코드를 이용해 텍스트 입력의 숫자 변환 과정을 체크한다
### b) testset의 임의 입력을 LongTensor로 변환해 학습 완료된 모델에 입력해보고, 결과가 어떠한지 체크한다 (하단의 출력 결과를 참조)

In [ ]:
input_text = testset[3].text
print(input_text)
print(TEXT.vocab[input_text[0]])

['the', 'mystery', 'and', 'its', 'solution', 'was', 'a', 'great', 'noir', 'conceit.', 'i', 'do', 'have', 'some', 'questions', 'though,', 'maybe', 'i', "wasn't", 'paying', 'enough', 'attention.<br', '/><br', '/>who', 'killed', 'the', 'neighbor', 'and', 'why?', 'who', 'killed', 'the', 'replacement', 'girl', 'and', 'why?<br', '/><br', '/>and', 'some', 'minor', 'quibbles,', 'they', 'should', 'have', 'shown', 'the', 'stopoff', 'at', 'the', 'hotel', 'for', 'the', 'switch.', 'not', 'that', 'they', 'should', 'have', 'shown', 'the', 'switch,', 'but', 'they', 'should', 'have', 'shown', 'jim', 'and', 'the', 'girl', 'going', 'in', 'the', 'hotel,', 'jim', 'going', 'to', 'the', 'bathroom,', 'coming', 'out', 'and', 'being', 'told', 'by', 'the', 'bartender', 'that', 'his', 'girlfriend', 'went', 'to', 'the', 'car', 'without', 'him.<br', '/><br', '/>then,', 'jim', 'getting', 'back', 'in', 'the', 'car', 'and', 'seeing', 'the', 'sleeping', 'woman,', 'and', 'little', 'girl', 'in', 'his', 'back', 'seat.<br'

['the', 'mystery', 'and', 'its', 'solution', 'was', 'a', 'great', 'noir', 'conceit.', 'i', 'do', 'have', 'some', 'questions', 'though,', 'maybe', 'i', "wasn't", 'paying', 'enough', 'attention.<br', '/><br', '/>who', 'killed', 'the', 'neighbor', 'and', 'why?', 'who', 'killed', 'the', 'replacement', 'girl', 'and', 'why?<br', '/><br', '/>and', 'some', 'minor', 'quibbles,', 'they', 'should', 'have', 'shown', 'the', 'stopoff', 'at', 'the', 'hotel', 'for', 'the', 'switch.', 'not', 'that', 'they', 'should', 'have', 'shown', 'the', 'switch,', 'but', 'they', 'should', 'have', 'shown', 'jim', 'and', 'the', 'girl', 'going', 'in', 'the', 'hotel,', 'jim', 'going', 'to', 'the', 'bathroom,', 'coming', 'out', 'and', 'being', 'told', 'by', 'the', 'bartender', 'that', 'his', 'girlfriend', 'went', 'to', 'the', 'car', 'without', 'him.<br', '/><br', '/>then,', 'jim', 'getting', 'back', 'in', 'the', 'car', 'and', 'seeing', 'the', 'sleeping', 'woman,', 'and', 'little', 'girl', 'in', 'his', 'back', 'seat.<br'